# Parkway Project Use Case 1: Write Off Cases Prediction

## DATA PREPARATION

### Load Data

#### Import packages

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_rows', 500)
import warnings
warnings.filterwarnings(action='once')
import pickle
from imblearn.over_sampling import SMOTE

In [3]:
import os
os.chdir('../')
print(os.getcwd())

/home/jupyter/temp25Oct-Mok-cfc9083c3125ca7ce3bf62e5/PRS-PM-ISY5002-GROUP5/SystemCode


In [4]:
%load_ext autoreload
%autoreload 2
from datapipeline import Datapipeline
dpl = Datapipeline()

#### Read data from file

In [5]:
file_dir = './Data Exploration/data/uc1/'
dict_X_train_file_paths = {
#    'GHL' : [file_dir + 'GHL_new_train_X_uc1.pkl'],
#    'MEH' : [file_dir + 'MEH_new_train_X_uc1.pkl'],
    'PEH' : [file_dir + 'PEH_new_train_X_uc1.pkl'],
#    'PNH' : [file_dir + 'PNH_new_train_X_uc1.pkl']
}
dict_y_train_file_paths = {
    'GHL' : file_dir + 'GHL_data_y_train.pkl',
    'MEH' : file_dir + 'MEH_data_y_train.pkl',
    'PEH' : file_dir + 'PEH_data_y_train.pkl',
    'PNH' : file_dir + 'PNH_data_y_train.pkl'
}
dict_X_test_file_paths = {
    'GHL' : [file_dir + 'GHL_new_test_X_uc1.pkl'],
    'MEH' : [file_dir + 'MEH_new_test_X_uc1.pkl'],
    'PEH' : [file_dir + 'PEH_new_test_X_uc1.pkl'],
    'PNH' : [file_dir + 'PNH_new_test_X_uc1.pkl']
}
dict_y_test_file_paths = {
    'GHL' : file_dir + 'GHL_data_y_test.pkl',
    'MEH' : file_dir + 'MEH_data_y_test.pkl',
    'PEH' : file_dir + 'PEH_data_y_test.pkl',
    'PNH' : file_dir + 'PNH_data_y_test.pkl'
}

In [6]:
dict_df_X_train = {}
dict_df_y_train = {}
dict_df_X_test = {}
dict_df_y_test = {}

for hosp in dict_X_train_file_paths:
    dict_df_X_train[hosp] = pd.concat([pd.read_pickle(file_path)
                                       for file_path in dict_X_train_file_paths[hosp]])
    print(f'X_train {hosp} {dict_df_X_train[hosp].shape}')
    
    dict_df_y_train[hosp] = pd.read_pickle(dict_y_train_file_paths[hosp])
    print(f'y_train {hosp} {dict_df_y_train[hosp].shape}')

    dict_df_X_test[hosp] = pd.concat([pd.read_pickle(file_path)
                                      for file_path in dict_X_test_file_paths[hosp]])
    print(f'X_test {hosp} {dict_df_X_test[hosp].shape}')

    dict_df_y_test[hosp] = pd.read_pickle(dict_y_test_file_paths[hosp])
    print(f'y_test {hosp} {dict_df_y_test[hosp].shape}')

X_train PEH (28604, 1037)
y_train PEH (28604,)
X_test PEH (9534, 1037)
y_test PEH (9534,)


In [7]:
def convert_y(df):
    df1 = dpl.bin_column(df.to_frame(), col='WRITE_OFF', bin_thresh = [500])
    df1 = pd.get_dummies(df1['bin_WRITE_OFF'])
    df1 = df1.drop(0, axis=1)
    df1.columns = ['WRITE_OFF_LABEL']
    return df1

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
for hosp in dict_df_y_train:
    dict_df_y_train[hosp] = convert_y(dict_df_y_train[hosp])
    dict_df_y_test[hosp] = convert_y(dict_df_y_test[hosp])

## DECISION TREE

### Decision Tree WITHOUT SMOTE

In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix  
from sklearn.tree import export_graphviz
from subprocess import call

In [10]:
criterion = 'entropy'
rand_seed = 0

In [11]:
for hosp in dict_df_X_train:
    model = DecisionTreeClassifier(criterion=criterion,random_state=rand_seed)
    model.fit(dict_df_X_train[hosp], dict_df_y_train[hosp])
    print(model)
    print(f"Accuracy on {hosp} training set without SMOTE decision tree: {model.score(dict_df_X_train[hosp], dict_df_y_train[hosp])}")
    print(f"Accuracy on {hosp} test set without SMOTE decision tree: {model.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")
    
    filename = file_dir + f'{hosp}_uc1_no_smote_dt_model.pkl'
    pickle.dump(model, open(filename, 'wb'))
    
    y_pred = model.predict(dict_df_X_test[hosp])
    print(hosp, 'decision tree\n', confusion_matrix(dict_df_y_test[hosp], y_pred))  
    print(hosp, 'decision tree\n', classification_report(dict_df_y_test[hosp], y_pred)) 

DecisionTreeClassifier(criterion='entropy', random_state=0)
Accuracy on PEH training set without SMOTE decision tree: 1.0
Accuracy on PEH test set without SMOTE decision tree: 0.998846234529054
PEH decision tree
 [[9523    4]
 [   7    0]]
PEH decision tree
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      9527
           1       0.00      0.00      0.00         7

    accuracy                           1.00      9534
   macro avg       0.50      0.50      0.50      9534
weighted avg       1.00      1.00      1.00      9534



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/PEH_uc1_no_smote_dt_model.pkl'>
  if __name__ == '__main__':


### Decision Tree WITH SMOTE

#### Decision Tree WITH SMOTE SS = 0.1 ~ 1.0

In [13]:
for hosp in dict_df_X_train:
    for sample in np.arange(0.1, 1.1, 0.1):
        sm_ss = SMOTE(random_state=55,sampling_strategy=sample)
        X_train_ss, y_train_ss = sm_ss.fit_sample(dict_df_X_train[hosp], dict_df_y_train[hosp])
        
        model = DecisionTreeClassifier(criterion=criterion,random_state=rand_seed)
        model.fit(X_train_ss, y_train_ss)
        print(model)
        print(f"Accuracy on {hosp} training set with {sample} SMOTE decision tree: {model.score(X_train_ss, y_train_ss)}")
        print(f"Accuracy on {hosp} test set with {sample} SMOTE decision tree: {model.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")
    
        filename = file_dir + f'{hosp}_uc1_{sample}_smote_dt_model.pkl'
        pickle.dump(model, open(filename, 'wb'))
        
        y_pred = model.predict(dict_df_X_test[hosp])
        print(hosp, 'decision tree \n', confusion_matrix(dict_df_y_test[hosp], y_pred))  
        print(hosp, 'decision tree \n', classification_report(dict_df_y_test[hosp], y_pred)) 

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


DecisionTreeClassifier(criterion='entropy', random_state=0)
Accuracy on PEH training set with 0.1 SMOTE decision tree: 1.0
Accuracy on PEH test set with 0.1 SMOTE decision tree: 0.9985315712187959
PEH decision tree 
 [[9520    7]
 [   7    0]]
PEH decision tree 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      9527
           1       0.00      0.00      0.00         7

    accuracy                           1.00      9534
   macro avg       0.50      0.50      0.50      9534
weighted avg       1.00      1.00      1.00      9534



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/PEH_uc1_0.1_smote_dt_model.pkl'>
  del sys.path[0]


DecisionTreeClassifier(criterion='entropy', random_state=0)
Accuracy on PEH training set with 0.2 SMOTE decision tree: 1.0
Accuracy on PEH test set with 0.2 SMOTE decision tree: 0.9985315712187959
PEH decision tree 
 [[9520    7]
 [   7    0]]
PEH decision tree 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      9527
           1       0.00      0.00      0.00         7

    accuracy                           1.00      9534
   macro avg       0.50      0.50      0.50      9534
weighted avg       1.00      1.00      1.00      9534



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/PEH_uc1_0.2_smote_dt_model.pkl'>
  del sys.path[0]


...

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/PEH_uc1_0.8_smote_dt_model.pkl'>
  del sys.path[0]


DecisionTreeClassifier(criterion='entropy', random_state=0)
Accuracy on PEH training set with 0.9 SMOTE decision tree: 1.0
Accuracy on PEH test set with 0.9 SMOTE decision tree: 0.9980071323683658
PEH decision tree 
 [[9515   12]
 [   7    0]]
PEH decision tree 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      9527
           1       0.00      0.00      0.00         7

    accuracy                           1.00      9534
   macro avg       0.50      0.50      0.50      9534
weighted avg       1.00      1.00      1.00      9534



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/PEH_uc1_0.9_smote_dt_model.pkl'>
  del sys.path[0]


DecisionTreeClassifier(criterion='entropy', random_state=0)
Accuracy on PEH training set with 1.0 SMOTE decision tree: 1.0
Accuracy on PEH test set with 1.0 SMOTE decision tree: 0.9982169079085379
PEH decision tree 
 [[9517   10]
 [   7    0]]
PEH decision tree 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      9527
           1       0.00      0.00      0.00         7

    accuracy                           1.00      9534
   macro avg       0.50      0.50      0.50      9534
weighted avg       1.00      1.00      1.00      9534



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/PEH_uc1_1.0_smote_dt_model.pkl'>
  del sys.path[0]


## LOGISTIC REGRESSION

### Logistic Regression Without SMOTE

In [14]:
from sklearn.linear_model import LogisticRegression

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
reg_strength = 0.01

In [16]:
for hosp in dict_df_X_train:
    model = LogisticRegression(C=reg_strength).fit(dict_df_X_train[hosp], dict_df_y_train[hosp])
    print(model)
    print(f"{hosp} Training set without SMOTE log reg score: {model.score(dict_df_X_train[hosp], dict_df_y_train[hosp])}")
    print(f"{hosp} Test set without SMOTE log reg score: {model.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")
    
    filename = file_dir + f'{hosp}_uc1_no_smote_logreg_model.pkl'
    pickle.dump(model, open(filename, 'wb'))
    
    y_pred = model.predict(dict_df_X_test[hosp])
    print(hosp, 'log reg \n', confusion_matrix(dict_df_y_test[hosp], y_pred))  
    print(hosp, 'log reg \n', classification_report(dict_df_y_test[hosp], y_pred)) 

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


LogisticRegression(C=0.01)
PEH Training set without SMOTE log reg score: 0.9992658369458817
PEH Test set without SMOTE log reg score: 0.9992657856093979
PEH log reg 
 [[9527    0]
 [   7    0]]
PEH log reg 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      9527
           1       0.00      0.00      0.00         7

    accuracy                           1.00      9534
   macro avg       0.50      0.50      0.50      9534
weighted avg       1.00      1.00      1.00      9534



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/PEH_uc1_no_smote_logreg_model.pkl'>
  
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Logistic Regression With SMOTE

#### Logistic Regression With SMOTE ss=0.1 ~ 1.0

In [17]:
for hosp in dict_df_X_train:
    for sample in np.arange(0.1, 1.1, 0.1):
        sm_ss = SMOTE(random_state=55,sampling_strategy=sample)
        X_train_ss, y_train_ss = sm_ss.fit_sample(dict_df_X_train[hosp], dict_df_y_train[hosp])
        
        model = LogisticRegression(C=reg_strength).fit(X_train_ss, y_train_ss)
        print(model)
        print(f"{hosp} Training set with {sample} SMOTE log reg score: {model.score(X_train_ss, y_train_ss)}")
        print(f"{hosp} Test set with {sample} SMOTE log reg score: {model.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")

        filename = file_dir + f'{hosp}_uc1_{sample}_smote_logreg_model.pkl'
        pickle.dump(model, open(filename, 'wb'))

        y_pred = model.predict(dict_df_X_test[hosp])
        print(hosp, 'log reg \n', confusion_matrix(dict_df_y_test[hosp], y_pred))  
        print(hosp, 'log reg \n', classification_report(dict_df_y_test[hosp], y_pred)) 

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


LogisticRegression(C=0.01)
PEH Training set with 0.1 SMOTE log reg score: 0.9681615776081425
PEH Test set with 0.1 SMOTE log reg score: 0.9950702748059577
PEH log reg 
 [[9487   40]
 [   7    0]]
PEH log reg 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      9527
           1       0.00      0.00      0.00         7

    accuracy                           1.00      9534
   macro avg       0.50      0.50      0.50      9534
weighted avg       1.00      1.00      1.00      9534



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/PEH_uc1_0.1_smote_logreg_model.pkl'>
  if sys.path[0] == '':
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


LogisticRegression(C=0.01)
PEH Training set with 0.2 SMOTE log reg score: 0.988891480552802
PEH Test set with 0.2 SMOTE log reg score: 0.9899307740717432
PEH log reg 
 [[9437   90]
 [   6    1]]
PEH log reg 
               precision    recall  f1-score   support

           0       1.00      0.99      0.99      9527
           1       0.01      0.14      0.02         7

    accuracy                           0.99      9534
   macro avg       0.51      0.57      0.51      9534
weighted avg       1.00      0.99      0.99      9534



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/PEH_uc1_0.2_smote_logreg_model.pkl'>
  if sys.path[0] == '':
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


...

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/PEH_uc1_0.8_smote_logreg_model.pkl'>
  if sys.path[0] == '':
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


LogisticRegression(C=0.01)
PEH Training set with 0.9 SMOTE log reg score: 0.984826443237271
PEH Test set with 0.9 SMOTE log reg score: 0.9687434445143697
PEH log reg 
 [[9234  293]
 [   5    2]]
PEH log reg 
               precision    recall  f1-score   support

           0       1.00      0.97      0.98      9527
           1       0.01      0.29      0.01         7

    accuracy                           0.97      9534
   macro avg       0.50      0.63      0.50      9534
weighted avg       1.00      0.97      0.98      9534



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/PEH_uc1_0.9_smote_logreg_model.pkl'>
  if sys.path[0] == '':
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


LogisticRegression(C=0.01)
PEH Training set with 1.0 SMOTE log reg score: 0.984710657056889
PEH Test set with 1.0 SMOTE log reg score: 0.9670652401929934
PEH log reg 
 [[9218  309]
 [   5    2]]
PEH log reg 
               precision    recall  f1-score   support

           0       1.00      0.97      0.98      9527
           1       0.01      0.29      0.01         7

    accuracy                           0.97      9534
   macro avg       0.50      0.63      0.50      9534
weighted avg       1.00      0.97      0.98      9534



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/PEH_uc1_1.0_smote_logreg_model.pkl'>
  if sys.path[0] == '':


## NAIVE BAYES

### Naive Bayes Without SMOTE

In [18]:
from sklearn.naive_bayes import GaussianNB

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [19]:
for hosp in dict_df_X_train:
    # Initiating the Gaussian Classifier
    model = GaussianNB()

    # Training your model 
    model.fit(dict_df_X_train[hosp], dict_df_y_train[hosp])
    print(model)
    filename = file_dir + f'{hosp}_uc1_no_smote_nb_model.pkl'
    pickle.dump(model, open(filename, 'wb'))

    # Score
    print(f"{hosp} Training set without SMOTE Naive Bayes score: {model.score(dict_df_X_train[hosp], dict_df_y_train[hosp])}")
    print(f"{hosp} Test set without SMOTE Naive Bayes score: {model.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")

    # Confusion Matrix
    y_pred = model.predict(dict_df_X_test[hosp])
    print(hosp, 'Naive Bayes \n', confusion_matrix(dict_df_y_test[hosp], y_pred))  
    print(hosp, 'Naive Bayes \n', classification_report(dict_df_y_test[hosp], y_pred)) 

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


GaussianNB()


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/PEH_uc1_no_smote_nb_model.pkl'>
  if __name__ == '__main__':


PEH Training set without SMOTE Naive Bayes score: 0.9468256187945742
PEH Test set without SMOTE Naive Bayes score: 0.9460876861757919
PEH Naive Bayes 
 [[9019  508]
 [   6    1]]
PEH Naive Bayes 
               precision    recall  f1-score   support

           0       1.00      0.95      0.97      9527
           1       0.00      0.14      0.00         7

    accuracy                           0.95      9534
   macro avg       0.50      0.54      0.49      9534
weighted avg       1.00      0.95      0.97      9534



### Naive Bayes With SMOTE

#### Naive Bayes Without SMOTE ss=0.1 ~ 1.0

In [20]:
for hosp in dict_df_X_train:
    for sample in np.arange(0.1, 1.1, 0.1):
        sm_ss = SMOTE(random_state=55,sampling_strategy=sample)
        X_train_ss, y_train_ss = sm_ss.fit_sample(dict_df_X_train[hosp], dict_df_y_train[hosp])
        
        # Initiating the Gaussian Classifier
        model = GaussianNB()
        
        # Training your model 
        model.fit(X_train_ss, y_train_ss)
        print(model)
        filename = file_dir + f'{hosp}_uc1_{sample}_smote_nb_model.pkl'
        pickle.dump(model, open(filename, 'wb'))

        # Score
        print(f"{hosp} Training set with {sample} SMOTE Naive Bayes score: {model.score(X_train_ss, y_train_ss)}")
        print(f"{hosp} Test set with {sample} SMOTE Naive Bayes score: {model.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")

        # Confusion Matrix
        y_pred = model.predict(dict_df_X_test[hosp])
        print(hosp, 'Naive Bayes\n', confusion_matrix(dict_df_y_test[hosp], y_pred))  
        print(hosp, 'Naive Bayes\n', classification_report(dict_df_y_test[hosp], y_pred)) 

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


GaussianNB()


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/PEH_uc1_0.1_smote_nb_model.pkl'>
  del sys.path[0]


PEH Training set with 0.1 SMOTE Naive Bayes score: 0.9516221374045801
PEH Test set with 0.1 SMOTE Naive Bayes score: 0.9460876861757919
PEH Naive Bayes
 [[9019  508]
 [   6    1]]
PEH Naive Bayes
               precision    recall  f1-score   support

           0       1.00      0.95      0.97      9527
           1       0.00      0.14      0.00         7

    accuracy                           0.95      9534
   macro avg       0.50      0.54      0.49      9534
weighted avg       1.00      0.95      0.97      9534



/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


...

PEH Training set with 0.9 SMOTE Naive Bayes score: 0.9719915293251082
PEH Test set with 0.9 SMOTE Naive Bayes score: 0.9460876861757919
PEH Naive Bayes
 [[9019  508]
 [   6    1]]
PEH Naive Bayes
               precision    recall  f1-score   support

           0       1.00      0.95      0.97      9527
           1       0.00      0.14      0.00         7

    accuracy                           0.95      9534
   macro avg       0.50      0.54      0.49      9534
weighted avg       1.00      0.95      0.97      9534



/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


GaussianNB()


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/PEH_uc1_1.0_smote_nb_model.pkl'>
  del sys.path[0]


PEH Training set with 1.0 SMOTE Naive Bayes score: 0.973392344832412
PEH Test set with 1.0 SMOTE Naive Bayes score: 0.9460876861757919
PEH Naive Bayes
 [[9019  508]
 [   6    1]]
PEH Naive Bayes
               precision    recall  f1-score   support

           0       1.00      0.95      0.97      9527
           1       0.00      0.14      0.00         7

    accuracy                           0.95      9534
   macro avg       0.50      0.54      0.49      9534
weighted avg       1.00      0.95      0.97      9534



## NEURAL NET MLP

### Neural Net MLP Without SMOTE, 2 Layers

In [21]:
from sklearn.neural_network import MLPClassifier  
from sklearn import metrics

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
list_hid_layers = [(128,128), (16,16,16), (32,32,32), (16,16,16,16), (32,32,32,32)]
max_iter = 1000
list_smote_sampling = [0.1, 0.2, 0.3, 0.5, 1.0]

#### Neural Net MLPClassifier without SMOTE

In [24]:
for hosp in dict_df_X_train:
    for hid_layers in list_hid_layers:
        model = MLPClassifier(hidden_layer_sizes=hid_layers, max_iter=max_iter,verbose=1)  
        model.fit(dict_df_X_train[hosp], dict_df_y_train[hosp])
        print(model)
        filename = file_dir + f'{hosp}_uc1_no_smote_mlp_{hid_layers}_model.pkl'
        pickle.dump(model, open(filename, 'wb'))

        predictions = model.predict(dict_df_X_test[hosp]) 

        # Score
        print(f"{hosp} Training set without SMOTE mlp {hid_layers} score: {model.score(dict_df_X_train[hosp], dict_df_y_train[hosp])}")
        print(f"{hosp} Test set without SMOTE mlp {hid_layers} score: {model.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")

        print(f"{hosp} Accuracy without SMOTE mlp {hid_layers}: ", metrics.accuracy_score(dict_df_y_test[hosp], predictions))
        print(hosp, f'mlp {hid_layers}\n', confusion_matrix(dict_df_y_test[hosp],predictions))  
        print(hosp, f'mlp {hid_layers}\n', classification_report(dict_df_y_test[hosp],predictions))  

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Iteration 1, loss = 0.05134183
Iteration 2, loss = 0.00640461
Iteration 3, loss = 0.00501225
Iteration 4, loss = 0.00431821
Iteration 5, loss = 0.00329276
Iteration 6, loss = 0.00288619
Iteration 7, loss = 0.00238296
Iteration 8, loss = 0.00218527
Iteration 9, loss = 0.00163663
Iteration 10, loss = 0.00126928
Iteration 11, loss = 0.00108314
Iteration 12, loss = 0.00120742
Iteration 13, loss = 0.00078403
Iteration 14, loss = 0.00103049
Iteration 15, loss = 0.00096872
Iteration 16, loss = 0.00037005
Iteration 17, loss = 0.00029468
Iteration 18, loss = 0.00023912
Iteration 19, loss = 0.00020172
Iteration 20, loss = 0.00016305
Iteration 21, loss = 0.00013985
Iteration 22, loss = 0.00012648
Iteration 23, loss = 0.00011591
Iteration 24, loss = 0.00010773
Iteration 25, loss = 0.00010199
Iteration 26, loss = 0.00009873
Iteration 27, loss = 0.00009253
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
MLPClassifier(hidden_layer_sizes=(128, 128), max_iter=1

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/PEH_uc1_no_smote_mlp_(128, 128)_model.pkl'>
  import sys


PEH Training set without SMOTE mlp (128, 128) score: 1.0
PEH Test set without SMOTE mlp (128, 128) score: 0.999160897839312
PEH Accuracy without SMOTE mlp (128, 128):  0.999160897839312
PEH mlp (128, 128)
 [[9526    1]
 [   7    0]]
PEH mlp (128, 128)
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      9527
           1       0.00      0.00      0.00         7

    accuracy                           1.00      9534
   macro avg       0.50      0.50      0.50      9534
weighted avg       1.00      1.00      1.00      9534



/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


...

PEH mlp (16, 16, 16, 16)
 [[9525    2]
 [   7    0]]
PEH mlp (16, 16, 16, 16)
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      9527
           1       0.00      0.00      0.00         7

    accuracy                           1.00      9534
   macro avg       0.50      0.50      0.50      9534
weighted avg       1.00      1.00      1.00      9534



/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Iteration 1, loss = 0.07489871
Iteration 2, loss = 0.00656377
Iteration 3, loss = 0.00561511
Iteration 4, loss = 0.00469598
Iteration 5, loss = 0.00393783
Iteration 6, loss = 0.00305565
Iteration 7, loss = 0.00254195
Iteration 8, loss = 0.00205536
Iteration 9, loss = 0.00180727
Iteration 10, loss = 0.00124315
Iteration 11, loss = 0.00145142
Iteration 12, loss = 0.00084438
Iteration 13, loss = 0.00085079
Iteration 14, loss = 0.00048752
Iteration 15, loss = 0.00058178
Iteration 16, loss = 0.00072462
Iteration 17, loss = 0.00051953
Iteration 18, loss = 0.00044436
Iteration 19, loss = 0.00228552
Iteration 20, loss = 0.00054927
Iteration 21, loss = 0.00082241
Iteration 22, loss = 0.00023036
Iteration 23, loss = 0.00013783
Iteration 24, loss = 0.00009855
Iteration 25, loss = 0.00008208
Iteration 26, loss = 0.00006950
Iteration 27, loss = 0.00006266
Iteration 28, loss = 0.00005744
Iteration 29, loss = 0.00005349
Iteration 30, loss = 0.00005103
Iteration 31, loss = 0.00004911
Iteration 32, los

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/PEH_uc1_no_smote_mlp_(32, 32, 32, 32)_model.pkl'>
  import sys


PEH Test set without SMOTE mlp (32, 32, 32, 32) score: 0.999056010069226
PEH Accuracy without SMOTE mlp (32, 32, 32, 32):  0.999056010069226
PEH mlp (32, 32, 32, 32)
 [[9525    2]
 [   7    0]]
PEH mlp (32, 32, 32, 32)
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      9527
           1       0.00      0.00      0.00         7

    accuracy                           1.00      9534
   macro avg       0.50      0.50      0.50      9534
weighted avg       1.00      1.00      1.00      9534



### Neural Net With SMOTE

In [25]:
list_hid_layers = [(16,16), (32,32), (64,64), (128,128), (256,256),
                   (16,16,16), (32,32,32), (64,64,64),
                   (16,16,16,16), (32,32,32,32)]
max_iter = 1000
list_smote_sampling = [0.1, 0.2, 0.3, 0.5, 1.0]

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### Neural Net Layers With SMOTE

In [26]:
for hosp in dict_df_X_train:
    for hid_layers in list_hid_layers:
        for sample in list_smote_sampling:
            sm_ss = SMOTE(random_state=55,sampling_strategy=sample)
            X_train_ss, y_train_ss = sm_ss.fit_sample(dict_df_X_train[hosp], dict_df_y_train[hosp])
        
            model = MLPClassifier(hidden_layer_sizes=hid_layers, max_iter=max_iter,verbose=1)  
            model.fit(X_train_ss, y_train_ss)  
            print(model)
            filename = file_dir + f'{hosp}_uc1_{sample}_smote_mlp_{hid_layers}_model.pkl'
            pickle.dump(model, open(filename, 'wb'))
            
            predictions = model.predict(dict_df_X_test[hosp]) 

            # Score
            print(f"{hosp} Training set {sample} SMOTE mlp {hid_layers} score: {model.score(X_train_ss, y_train_ss)}")
            print(f"{hosp} Test set {sample} SMOTE mlp {hid_layers} score: {model.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")

            print(f"{hosp} Accuracy: ", metrics.accuracy_score(dict_df_y_test[hosp], predictions))
            print(hosp, f'mlp {hid_layers}\n', confusion_matrix(dict_df_y_test[hosp],predictions))  
            print(hosp, f'mlp {hid_layers}\n', classification_report(dict_df_y_test[hosp],predictions))  

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Iteration 1, loss = 0.32803756
Iteration 2, loss = 0.05552681
Iteration 3, loss = 0.01462244
Iteration 4, loss = 0.00691688
Iteration 5, loss = 0.00437374
Iteration 6, loss = 0.00303931
Iteration 7, loss = 0.00208975
Iteration 8, loss = 0.00155727
Iteration 9, loss = 0.00116680
Iteration 10, loss = 0.00094688
Iteration 11, loss = 0.00073337
Iteration 12, loss = 0.00063759
Iteration 13, loss = 0.00052500
Iteration 14, loss = 0.00044741
Iteration 15, loss = 0.00037893
Iteration 16, loss = 0.00033267
Iteration 17, loss = 0.00029401
Iteration 18, loss = 0.00022658
Iteration 19, loss = 0.00021326
Iteration 20, loss = 0.00017124
Iteration 21, loss = 0.00015183
Iteration 22, loss = 0.00013100
Iteration 23, loss = 0.00012194
Iteration 24, loss = 0.00010953
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
MLPClassifier(hidden_layer_sizes=(16, 16), max_iter=1000, verbose=1)
PEH Training set 0.1 SMOTE mlp (16, 16) score: 1.0
PEH Test set 0.1 SMOTE mlp (16,

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/PEH_uc1_0.1_smote_mlp_(16, 16)_model.pkl'>
  # This is added back by InteractiveShellApp.init_path()


PEH mlp (16, 16)
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      9527
           1       0.00      0.00      0.00         7

    accuracy                           1.00      9534
   macro avg       0.50      0.50      0.50      9534
weighted avg       1.00      1.00      1.00      9534



/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


...

PEH Training set 0.5 SMOTE mlp (32, 32, 32, 32) score: 1.0
PEH Test set 0.5 SMOTE mlp (32, 32, 32, 32) score: 0.99895112229914
PEH Accuracy:  0.99895112229914
PEH mlp (32, 32, 32, 32)
 [[9524    3]
 [   7    0]]
PEH mlp (32, 32, 32, 32)
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      9527
           1       0.00      0.00      0.00         7

    accuracy                           1.00      9534
   macro avg       0.50      0.50      0.50      9534
weighted avg       1.00      1.00      1.00      9534



/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Iteration 1, loss = 0.09619092
Iteration 2, loss = 0.00152470
Iteration 3, loss = 0.00037108
Iteration 4, loss = 0.00023076
Iteration 5, loss = 0.00017168
Iteration 6, loss = 0.00013019
Iteration 7, loss = 0.00005048
Iteration 8, loss = 0.00004750
Iteration 9, loss = 0.00004561
Iteration 10, loss = 0.00004412
Iteration 11, loss = 0.00004295
Iteration 12, loss = 0.00004192
Iteration 13, loss = 0.00004101
Iteration 14, loss = 0.00004018
Iteration 15, loss = 0.00003938
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
MLPClassifier(hidden_layer_sizes=(32, 32, 32, 32), max_iter=1000, verbose=1)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/PEH_uc1_1.0_smote_mlp_(32, 32, 32, 32)_model.pkl'>
  # This is added back by InteractiveShellApp.init_path()


PEH Training set 1.0 SMOTE mlp (32, 32, 32, 32) score: 1.0
PEH Test set 1.0 SMOTE mlp (32, 32, 32, 32) score: 0.9987413467589679
PEH Accuracy:  0.9987413467589679
PEH mlp (32, 32, 32, 32)
 [[9522    5]
 [   7    0]]
PEH mlp (32, 32, 32, 32)
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      9527
           1       0.00      0.00      0.00         7

    accuracy                           1.00      9534
   macro avg       0.50      0.50      0.50      9534
weighted avg       1.00      1.00      1.00      9534



## KNN

In [27]:
# Be careful when running this, about 2 full days are needed to run this!!!
from sklearn.neighbors import KNeighborsClassifier

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
for hosp in dict_df_X_train:
    test_scores = []
    train_scores = []

    for i in range(1,15):

        knn = KNeighborsClassifier(i)
        knn.fit(dict_df_X_train[hosp],dict_df_y_train[hosp])

        train_scores.append(knn.score(dict_df_X_train[hosp],dict_df_y_train[hosp]))
        test_scores.append(knn.score(dict_df_X_test[hosp],dict_df_y_test[hosp]))

    ## score that comes from testing on the same datapoints that were used for training
    max_train_score = max(train_scores)
    train_scores_ind = [i for i, v in enumerate(train_scores) if v == max_train_score]
    print(f'{hosp} Max train score {max_train_score*100} % and k = {list(map(lambda x: x+1, train_scores_ind))}')

    ## score that comes from testing on the datapoints that were split in the beginning to be used for testing solely
    max_test_score = max(test_scores)
    test_scores_ind = [i for i, v in enumerate(test_scores) if v == max_test_score]
    print(f'{hosp} Max test score {max_test_score*100} % and k = {list(map(lambda x: x+1, test_scores_ind))}')

    plt.figure(figsize=(12,5))
    plt.plot(range(1,15),train_scores,marker='*',label='Train Score')
    plt.plot(range(1,15),test_scores,marker='o',label='Test Score')
    plt.title(f'{hosp} KNN')
    plt.tight_layout()
    plt.show()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
# From the above, the best KNN is at k=8
#Setup a knn classifier with k neighbors
best_knn = KNeighborsClassifier(8)

### KNN without SMOTE

In [ ]:
n_neighbours = 8
for hosp in dict_df_X_train:
    # Fit the KNN model with training data and score the test data
    best_knn = KNeighborsClassifier(n_neighbours)
    
    best_knn.fit(dict_df_X_train[hosp],dict_df_y_train[hosp])
    best_knn.score(dict_df_X_test[hosp],dict_df_y_test[hosp])
    
    # save the model to disk
    filename = f'{hosp}_best_knn_without_smote.pkl'
    pickle.dump(best_knn, open(filename, 'wb'))
    
    y_pred = best_knn.predict(dict_df_X_test[hosp])
    # Score for Best KNN, No SMOTE
    print(f"{hosp} Training set without SMOTE KNN({n_neighbours}) score: {best_knn.score(dict_df_X_train[hosp], dict_df_y_train[hosp])}")
    print(f"{hosp} Test set without SMOTE KNN({n_neighbours}) score: {best_knn.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")

    print(f"{hosp} Accuracy without SMOTE KNN({n_neighbours}): {metrics.accuracy_score(dict_df_y_test[hosp], y_pred)}")
    print(hosp, f'KNN({n_neighbours})\n', confusion_matrix(dict_df_y_test[hosp],y_pred))  
    print(hosp, f'KNN({n_neighbours})\n', classification_report(dict_df_y_test[hosp],y_pred))  

### KNN With SMOTE

#### KNN With SMOTE ss=0.1

In [ ]:
n_neighbours = 8
list_smote_sampling = [0.1, 0.5, 1.0]
for hosp in dict_df_X_train:
    for sample in list_smote_sampling:
        sm_ss = SMOTE(random_state=55,sampling_strategy=sample)
        X_train_ss, y_train_ss = sm_ss.fit_sample(dict_df_X_train[hosp], dict_df_y_train[hosp])
        
        # Fit the KNN model with training data and score the test data
        best_knn = KNeighborsClassifier(n_neighbours)    
        best_knn.fit(X_train_ss,y_train_ss)
        #best_knn.score(dict_df_X_test[hosp],dict_df_y_test[hosp])
        
        # save the model to disk
        filename = f'{hosp}_best_knn_{sample}_smote.pkl'
        pickle.dump(best_knn, open(filename, 'wb'))
        
        y_pred = best_knn.predict(dict_df_X_test[hosp])
        # Score for Best KNN, with SMOTE
        print(f"{hosp} Training set {sample} SMOTE KNN({n_neighbours}) score: {best_knn.score(X_train_ss, y_train_ss)}")
        print(f"{hosp} Test set {sample} SMOTE KNN({n_neighbours}) score: {best_knn.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")

        print(f"{hosp} Accuracy {sample} SMOTE KNN({n_neighbours}): ", metrics.accuracy_score(dict_df_y_test[hosp], y_pred))
        print(confusion_matrix(dict_df_y_test[hosp],y_pred))  
        print(classification_report(dict_df_y_test[hosp],y_pred)) 